In [2]:
import os
import pandas as pd
import numpy as np
os.getcwd()

'/Users/siqisun/Documents/GitHub/CMM'

# 1. read 4 datasets

In [6]:
claims = pd.read_csv("dim_claims.csv")
claims.index = claims["dim_claim_id"]
print(claims.shape)
claims.head()

(1335576, 5)


,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
dim_claim_id,,,,,
1,1,417380,A,75.0,0
2,2,999001,A,NaN,1
3,3,417740,A,76.0,0
4,4,999001,A,NaN,1
5,5,417740,A,NaN,1


In [26]:
date = pd.read_csv("dim_date.csv")
date.index = date["dim_date_id"]
print(date.shape)
date.head()

(1520, 9)


,dim_date_id,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday
dim_date_id,,,,,,,,,
1,1,2017-01-01,2017,1,1,1,0,0,1
2,2,2017-01-02,2017,1,2,2,1,0,0
3,3,2017-01-03,2017,1,3,3,1,1,0
4,4,2017-01-04,2017,1,4,4,1,1,0
5,5,2017-01-05,2017,1,5,5,1,1,0


In [3]:
pa = pd.read_csv("dim_pa.csv")
pa.index = pa["dim_pa_id"]
print(pa.shape)
pa.head()

(555951, 5)


,dim_pa_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved
dim_pa_id,,,,,
1,1,1,1,0,1
2,2,1,0,0,1
3,3,0,0,1,1
4,4,1,1,0,1
5,5,0,1,0,1


In [5]:
bridge = pd.read_csv("bridge.csv")
bridge.index = bridge["dim_claim_id"]
print(bridge.shape)
bridge.head()

(1335576, 3)


,dim_claim_id,dim_pa_id,dim_date_id
dim_claim_id,,,
1,1,1.0,1
2,2,NaN,1
3,3,2.0,1
4,4,NaN,1
5,5,NaN,1


# 2. merge claims, pa and date data with bridge index

In [8]:
#merge claims info with bridge info
summary = pd.concat([bridge, claims], axis = 1)
summary.head()

,dim_claim_id,dim_pa_id,dim_date_id,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
dim_claim_id,,,,,,,,
1,1,1.0,1,1,417380,A,75.0,0
2,2,NaN,1,2,999001,A,NaN,1
3,3,2.0,1,3,417740,A,76.0,0
4,4,NaN,1,4,999001,A,NaN,1
5,5,NaN,1,5,417740,A,NaN,1


In [ ]:
#merge pa info
summary.index = summary["dim_pa_id"]
summary[pa.columns[1:]] = np.nan
summary.head()

In [22]:
for i in summary.index:
    if i in pa.index:
        for j in pa.columns[1:]:
            summary.loc[i, j] = pa.loc[i, j]

In [23]:
summary.head()

,dim_claim_id,dim_pa_id,dim_date_id,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved,correct_diagnosis,tried_and_failed,contraindication,pa_approved
dim_pa_id,,,,,,,,,,,,
1.0,1,1.0,1,1,417380,A,75.0,0,1.0,1.0,0.0,1.0
NaN,2,NaN,1,2,999001,A,NaN,1,NaN,NaN,NaN,NaN
2.0,3,2.0,1,3,417740,A,76.0,0,1.0,0.0,0.0,1.0
NaN,4,NaN,1,4,999001,A,NaN,1,NaN,NaN,NaN,NaN
NaN,5,NaN,1,5,417740,A,NaN,1,NaN,NaN,NaN,NaN


In [ ]:
#merge date info
summary.index = summary["dim_date_id"]
summary[date.columns[1:]] = np.nan
summary.head()

In [32]:
summary.loc[:, date.columns[1:]] = summary["dim_date_id"].apply(lambda x: date.loc[x, date.columns[1:]])

In [33]:
summary = summary.drop(["dim"])
summary.head()

,dim_claim_id,dim_pa_id,dim_date_id,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved,correct_diagnosis,tried_and_failed,contraindication,pa_approved,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday
dim_date_id,,,,,,,,,,,,,,,,,,,,
1,1,1.0,1,1,417380,A,75.0,0,1.0,1.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1
1,2,NaN,1,2,999001,A,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
1,3,2.0,1,3,417740,A,76.0,0,1.0,0.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1
1,4,NaN,1,4,999001,A,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
1,5,NaN,1,5,417740,A,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1


In [34]:
summary.to_csv("summary.csv", index = False) #just noticed dim_claim_id is duplicated

In [ ]:
summary = pd.read_csv("summary.csv")
summary = summary.drop(["dim_claim_id.1"], axis=1)
summary.to_csv("summary.csv", index = False)

In [9]:
summary.columns

Index(['dim_claim_id', 'dim_pa_id', 'dim_date_id', 'bin', 'drug',
       'reject_code', 'pharmacy_claim_approved', 'correct_diagnosis',
       'tried_and_failed', 'contraindication', 'pa_approved', 'date_val',
       'calendar_year', 'calendar_month', 'calendar_day', 'day_of_week',
       'is_weekday', 'is_workday', 'is_holiday'],
      dtype='object')